# Tarefa 1 - Preparação do Dataset do Occipital Lobe

In [1]:
! pip install researchpy

In [2]:
import os
import random
import pandas as pd
import sklearn as np
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [3]:
def set_seed(seed: int):
    random.seed(seed) # Python
    np.random.seed(seed)  # Numpy, é o gerador utilizado pelo sklearn
    os.environ["PYTHONHASHSEED"] = str(seed)  # sistema operativo

# Fixar a seed
set_seed(2023)

In [4]:
df_treino = pd.read_csv('./datasets/train_radiomics_occipital_CONTROL.csv')
pd.options.display.max_columns = None

In [5]:
df_treino.shape

(305, 2181)

In [6]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 2181 entries, ID to Transition
dtypes: float64(2014), int64(147), object(20)
memory usage: 5.1+ MB


### Colunas de valor único

Quando uma coluna contém o mesmo valor para todas as 305, esta não ajuda a distinguir entre as observações e não contribui para a construção de modelos preditivos, análises estatísticas, ou para entender padrões nos dados. Estas colunas podem ser consideradas redundantes e ocupam espaço desnecessário, o que também pode prejudicar a eficiência computacional ao aumentar o tempo de processamento. Com isto dito, em baixo pesquisamos todas essas colunas e eliminamos essas mesmo:

In [7]:
constant_columns_treino = df_treino.columns[df_treino.nunique() == 1]
print(constant_columns_treino)

Index(['diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy',
       'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet',
       'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings',
       'diagnostics_Configuration_EnabledImageTypes',
       'diagnostics_Image-original_Dimensionality',
       'diagnostics_Image-original_Spacing', 'diagnostics_Image-original_Size',
       ...
       'lbp-3D-m2_glszm_GrayLevelNonUniformityNormalized',
       'lbp-3D-m2_glszm_GrayLevelVariance',
       'lbp-3D-m2_glszm_HighGrayLevelZoneEmphasis',
       'lbp-3D-m2_glszm_LowGrayLevelZoneEmphasis',
       'lbp-3D-m2_glszm_SizeZoneNonUniformity', 'lbp-3D-m2_ngtdm_Busyness',
       'lbp-3D-m2_ngtdm_Coarseness', 'lbp-3D-m2_ngtdm_Complexity',
       'lbp-3D-m2_ngtdm_Contrast', 'lbp-3D-m2_ngtdm_Strength'],
      dtype='object', length=147)


In [8]:
df_treino = df_treino.drop(constant_columns_treino, axis=1)

In [9]:
df_treino.shape

(305, 2034)

In [10]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 2034 entries, ID to Transition
dtypes: float64(2001), int64(24), object(9)
memory usage: 4.7+ MB


Depois de feita a eliminação das colunas redundantes, através dos comando ***shape***, verificamos que eliminamos ***147 colunas redundantes*** do dataset.
Com o comando ***info()***, verificamos para o dataset, que dessas 147 colunas, 13 eram do tipo ***float64***, 123 eram do tipo ***int64*** e 11 eram do tipo ***object***.

### Análise de colunas do tipo ***object***

Consideramos que as seguintes colunas a eliminar, não apresentam dados necessários para a previsão do atributo-objetivo, ***Transition:***
- ***ID:*** porque representa apenas o ID das imagens obtidas.
- ***Image e Mask:*** porque representa apenas a localização dos diferentes scans.
- ***diagnostics_Image-original_Hash:*** porque representa um hash único para a imagem original, usado para verificar a integridade da imagem ou identificar duplicates, mas para análises quantitativas não é importante.
- ***diagnostics_Mask-original_Hash:*** porque representa um hash único da mask original, utilizado para verificar a integridade da mask. Para análises quantitativas não é importante.

In [11]:
object_columns_treino = df_treino.select_dtypes(include='object')
print(object_columns_treino.head())

           ID                                              Image  \
0  006_S_0681  /notebooks/disk2/DS2_FreeSurfer/ADNI_006_S_068...   
1  941_S_1203  /notebooks/disk2/DS2_FreeSurfer/ADNI_941_S_120...   
2  011_S_0003  /notebooks/disk2/DS2_FreeSurfer/ADNI_011_S_000...   
3  057_S_0779  /notebooks/disk2/DS2_FreeSurfer/ADNI_057_S_077...   
4  033_S_0920  /notebooks/disk2/DS2_FreeSurfer/ADNI_033_S_092...   

                                                Mask  \
0  /notebooks/disk2/DS2_FreeSurfer/ADNI_006_S_068...   
1  /notebooks/disk2/DS2_FreeSurfer/ADNI_941_S_120...   
2  /notebooks/disk2/DS2_FreeSurfer/ADNI_011_S_000...   
3  /notebooks/disk2/DS2_FreeSurfer/ADNI_057_S_077...   
4  /notebooks/disk2/DS2_FreeSurfer/ADNI_033_S_092...   

            diagnostics_Image-original_Hash  \
0  b5d774a32163a7ee822d42a07808a787f8687f56   
1  397042d736bd790b7880b372b1749ff424f89cbe   
2  84d679a88812c4aaf03a6d99f00c913b2f64506f   
3  168f330d2ca3f097146e5d041f33b40672d230df   
4  ea5f291ea107dfda

In [12]:
df_treino = df_treino.drop('ID', axis=1)
df_treino = df_treino.drop('Image', axis=1)
df_treino = df_treino.drop('Mask', axis=1)
df_treino = df_treino.drop('diagnostics_Image-original_Hash', axis=1)
df_treino = df_treino.drop('diagnostics_Mask-original_Hash', axis=1)
df_treino = df_treino.drop('diagnostics_Mask-original_CenterOfMass', axis=1)

In [13]:
df_treino.shape

(305, 2028)

In [14]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 2028 entries, diagnostics_Image-original_Mean to Transition
dtypes: float64(2001), int64(24), object(3)
memory usage: 4.7+ MB


De seguida, vamos analisar as colunas restantes do tipo ***object***.

In [15]:
object_columns_treino = df_treino.select_dtypes(include='object')
print(object_columns_treino.head())

  diagnostics_Mask-original_BoundingBox  \
0             (46, 99, 76, 43, 51, 105)   
1            (28, 137, 77, 48, 44, 100)   
2            (21, 109, 72, 46, 52, 112)   
3             (30, 85, 70, 50, 55, 110)   
4            (29, 105, 77, 47, 45, 108)   

         diagnostics_Mask-original_CenterOfMassIndex Transition  
0  (63.015277297359624, 126.84203851031938, 127.2...      CN-CN  
1  (48.72254303164908, 160.27007218212105, 122.57...      CN-CN  
2  (40.64246922519913, 139.6085716871832, 126.311...      AD-AD  
3  (49.16798692317364, 115.65732008885536, 127.28...     CN-MCI  
4  (47.38606338946533, 131.73039632590576, 133.67...      CN-CN  


In [16]:
# Separar os elementos dos tuplos em colunas individuais
bbox_cols = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'bbox_x3', 'bbox_y3']
com_cols = ['com_x', 'com_y', 'com_z']

df_treino[bbox_cols] = pd.DataFrame(df_treino['diagnostics_Mask-original_BoundingBox'].apply(eval).tolist(), index=df_treino.index)
df_treino[com_cols] = pd.DataFrame(df_treino['diagnostics_Mask-original_CenterOfMassIndex'].apply(eval).tolist(), index=df_treino.index)

# Remover as colunas originais
df_treino = df_treino.drop('diagnostics_Mask-original_BoundingBox', axis = 1)
df_treino = df_treino.drop('diagnostics_Mask-original_CenterOfMassIndex', axis=1)

# Explicação das colunas
# diagnostics_Mask-original_BoundingBox: Contém as coordenadas dos vértices da caixa delimitadora (bounding box).
# diagnostics_Mask-original_CenterOfMassIndex: Contém as coordenadas do centro de massa.

# Verificar a existência de valores ausentes
missing_values = df_treino.isnull().sum()

if missing_values.any():
    print('Sim')
    
    # Imprimir os nomes das colunas com valores ausentes
    cols_with_missing_values = missing_values[missing_values > 0].index.tolist()
    print("Colunas com valores ausentes:", cols_with_missing_values)
else:
    print('Não')

Não


In [17]:
# Imprimir as primeiras linhas das colunas especificadas
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'bbox_x3', 'bbox_y3', 'com_x', 'com_y', 'com_z']
print(df_treino[columns].head())

   bbox_x1  bbox_y1  bbox_x2  bbox_y2  bbox_x3  bbox_y3      com_x  \
0       46       99       76       43       51      105  63.015277   
1       28      137       77       48       44      100  48.722543   
2       21      109       72       46       52      112  40.642469   
3       30       85       70       50       55      110  49.167987   
4       29      105       77       47       45      108  47.386063   

        com_y       com_z  
0  126.842039  127.218264  
1  160.270072  122.572238  
2  139.608572  126.311052  
3  115.657320  127.281278  
4  131.730396  133.674434  


In [18]:
df_treino.shape

(305, 2035)

In [19]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 2035 entries, diagnostics_Image-original_Mean to com_z
dtypes: float64(2004), int64(30), object(1)
memory usage: 4.7+ MB


Eliminamos as seguintes colunas, ***diagnostics_Mask-original_BoundingBox', diagnostics_Mask-original_CenterOfMassIndex***, e criamos novas com cada um dos elementos de coordenadas dos tuplos ***['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'bbox_x3', 'bbox_y3', 'com_x', 'com_y', 'com_z'].***

### Remover colunas duplicadas

In [20]:
# Verificar se há colunas duplicadas em df_treino
duplicated_columns_treino = df_treino.T.duplicated(keep=False)

# Exibir resultado para df_treino
if duplicated_columns_treino.any():
    print("Ainda existem colunas duplicadas em df_treino:")
    duplicate_column_names_treino = df_treino.columns[duplicated_columns_treino]
    print(f"Número de colunas duplicadas em df_treino: {len(duplicate_column_names_treino)}")
else:
    print("Não há mais colunas duplicadas em df_treino.")

Ainda existem colunas duplicadas em df_treino:
Número de colunas duplicadas em df_treino: 166


In [21]:
def remove_duplicated_columns(df):
    # Identify duplicated columns (excluding the first occurrence)
    duplicated_columns = df.T.duplicated(keep='first')
    
    # Get the list of duplicate column names to remove
    columns_to_remove = df.columns[duplicated_columns]
    
    # Drop the duplicate columns
    df_cleaned = df.drop(columns=columns_to_remove)
    
    # Total number of columns removed
    total_removed = len(columns_to_remove)
    
    print(f"Total duplicated columns removed: {total_removed}")
    
    return df_cleaned, total_removed

# Usage
df_treino_cleaned, total_removed = remove_duplicated_columns(df_treino)
print(f"Dimensions after removal: {df_treino_cleaned.shape}")

Total duplicated columns removed: 115
Dimensions after removal: (305, 1920)


### Converter de float64 e int64 para float32 e int32

Em seguida, vamos tratar das conversões dos ints e floats do dataset de treino.

In [22]:
float_features = df_treino.select_dtypes(include='float')
int_features = df_treino.select_dtypes(include='int')

df_treino[float_features.columns] = df_treino[float_features.columns].astype(np.float32)
df_treino[int_features.columns] = df_treino[int_features.columns].astype(np.int32)
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 2035 entries, diagnostics_Image-original_Mean to com_z
dtypes: float32(2004), int32(30), object(1)
memory usage: 2.4+ MB


### Exportar os datasets resultantes

In [23]:
# Criar a pasta se ela não existir
output_folder = 'data_mod_occ'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Exportar o DataFrame para um arquivo CSV na pasta criada
output_file = os.path.join(output_folder, 'dataset_treino.csv')
df_treino.to_csv(output_file, index=False)

print(f"Dataset final de treino limpo exportado para {output_file}")

Dataset final de treino limpo exportado para data_mod_occ/dataset_treino.csv
